# Initial Setup

In [1]:
import os
import tensorflow

2024-05-11 05:56:04.212199: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-11 05:56:04.214250: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 05:56:04.240800: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 05:56:04.240833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 05:56:04.241515: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# LabelMaps

## Creacion de Labelmaps

In [ ]:
!python ./csv_to_labelmap.py

# TFRecords

## Train Model

In [ ]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/train/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/train" train.record

## Test Model

In [ ]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/test/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/test" test.record

## Valid Model

In [ ]:
!python ./generate_tfRecord.py "Playing Cards.v4-yolov8n.tensorflow/valid/_annotations.csv" label_map.pbtxt "Playing Cards.v4-yolov8n.tensorflow/valid" valid.record

# MLModel

## Libraries

In [2]:
import wget
import tarfile
import os
import re

## Enviroment Settings

In [3]:
#Can be configured
DATASET_DIRECTORY = "/tf/dataset/workspaceCards/"
OBJECT_DETECTION_DIRECTORY = "/tf/tensorflow/models/research/object_detection/"

MODEL_NAME = "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8"
MODEL_LINK = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz"
MODEL_CONFIG = "ssd_mobilenet_v2_pets_keras.config"
MODELS_FOLDER = OBJECT_DETECTION_DIRECTORY+"samples/configs/"
MODEL_FILEPATH = os.path.join(MODELS_FOLDER, MODEL_CONFIG)

In [4]:
FINE_CHECKPOINT = DATASET_DIRECTORY + "models/" + MODEL_NAME + "/checkpoint/cpkt-0"
TRAIN_TFRECORD = DATASET_DIRECTORY + "train.record"
TEST_TFRECORD = DATASET_DIRECTORY + "test.record"
LABEL_MAP = DATASET_DIRECTORY + "label_map.pbtxt"

In [5]:
BATCH_SIZE = 12
NUM_STEPS = 10000
NUM_CLASSES = 52
NUM_EVAL_STEPS = 50

## Downloading ML Model

In [ ]:
wget.download(MODEL_LINK, out='./models')

## Unzip MLModel

In [ ]:
tar = tarfile.open('./models/' + MODEL_NAME + ".tar.gz")
tar.extractall('./models/')
tar.close()

## Modify config file


In [7]:
with open(MODEL_FILEPATH) as f:
    s = f.read()

with open(MODEL_FILEPATH, 'w') as f:
        # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(FINE_CHECKPOINT), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(TRAIN_TFRECORD), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(TEST_TFRECORD), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(LABEL_MAP), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(BATCH_SIZE), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(NUM_STEPS), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(NUM_CLASSES), s)
    f.write(s)

In [8]:
!cat {MODEL_FILEPATH}

# An untested config for Keras SSD with MobileNetV2 configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 52
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
      

## Training model

In [10]:
!python /tf/tensorflow/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={MODEL_FILEPATH} \
    --model_dir = {} \
    --num_train_steps={NUM_STEPS} \
    --alsologtostderr

2024-05-11 05:59:04.162441: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-11 05:59:04.163909: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 05:59:04.189980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 05:59:04.190024: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 05:59:04.190918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to